# Exploratory Data Analysis of Heavy Truck J1939 data

Ce notebook va servir de base dans l'analyse des données du `Heavy Truck`. Ces données reprennent une payload que nous pouvons retrouver avec le protocole **bus CAN J1939**.

**Payload d'un message sous protocole J1939:**

![payload j1939](./images/j1939-payload.png)

:warning: Attention à bien écouter Vendredi 26 la présentation de `Simon Bellemare` sur les données **bus CAN J1939** !

In [ ]:
# Ignorez ce bout de code

import numpy as np
from PIL import Image


def remove_white_background_and_crop(image_path: str, output_path: str):
    """
    Removes white background from a JPG image, crops it to the content, 
    and saves it as a PNG (to support transparency).
    """
    img = Image.open(image_path).convert("RGBA")
    data = np.array(img)

    red, green, blue, alpha = data.T
    white_areas = (red > 240) & (green > 240) & (blue > 240)
    
    data[..., 3][white_areas.T] = 0
    
    img_transparent = Image.fromarray(data)
    
    bbox = img_transparent.getbbox()
    if bbox:
        img_cropped = img_transparent.crop(bbox)
        img_cropped.save(output_path)
        return img_cropped
    else:
        print("Image is completely empty after removing background.")
        return None

remove_white_background_and_crop("C:\\Users\\baske\\projects\\CANlock\\CANlock\\data\\canbus_canlock.jpg", "output_image.png")

# Import libraries

In [ ]:
import plotly.express as px

from canlock.db.database import get_session, init_db
from canlock.decoder import SessionDecoder

# Comment obtenir les signaux d'une session ?

In [ ]:
# Aussi simple que ça !
init_db()

with get_session() as session:
    decoder = SessionDecoder(db=session)
    sessions = decoder.list_sessions()

df_session = decoder.decode(session_id=sessions[12].id)
df_session.set_index("timestamp", inplace=True)

In [ ]:
df_session

In [ ]:
px.line(df_session["ENGINE_SPEED"].dropna().reset_index(drop=False), x="timestamp", y="ENGINE_SPEED", title="Engine speed during the session")